# 실전 문제 해결 (과소적합)

In [1]:
import tensorflow as tf
import numpy as np

## 데이터셋 준비


In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

## 모델 정의

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.dense3 = tf.keras.layers.Dense(128, activation='relu')
        self.dense4 = tf.keras.layers.Dense(256, activation='relu')
        self.dense5 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.dense5(x)

## 모델 학습

In [4]:
EPOCHS = 30

In [5]:
model = MyModel()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/30
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5201 - accuracy: 0.8117 - val_loss: 0.4448 - val_accuracy: 0.8362
Epoch 2/30
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3887 - accuracy: 0.8594 - val_loss: 0.3933 - val_accuracy: 0.8529
Epoch 3/30
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3563 - accuracy: 0.8687 - val_loss: 0.3777 - val_accuracy: 0.8595
Epoch 4/30
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3360 - accuracy: 0.8749 - val_loss: 0.3975 - val_accuracy: 0.8607
Epoch 5/30
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3184 - accuracy: 0.8821 - val_loss: 0.3737 - val_accuracy: 0.8663
Epoch 6/30
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3043 - accuracy: 0.8867 - val_loss: 0.3659 - val_accuracy: 0.8682
Epoch 7/30
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2934 - accuracy: 0.8906 - val_loss: 0.3522 - val_accuracy:

## 모델 수정 - DenseNet

In [8]:
class MyModel_DN(tf.keras.Model):
    def __init__(self):
        super(MyModel_DN, self).__init__()
        
        self.flatten = tf.keras.layers.Flatten()
        
        # dense - BN - relu
        self.dense1 = tf.keras.layers.Dense(32, use_bias=False)
        self.batch1 = tf.keras.layers.BatchNormalization()
        
        self.batch2 = tf.keras.layers.BatchNormalization()
        self.dense2 = tf.keras.layers.Dense(32, use_bias=False)  # 32 + 32 -> 64
        
        self.batch3 = tf.keras.layers.BatchNormalization()
        self.dense3 = tf.keras.layers.Dense(64, use_bias=False)  # 64 + 64 -> 128
        
        self.batch4 = tf.keras.layers.BatchNormalization()
        self.dense4 = tf.keras.layers.Dense(128, use_bias=False)
        
        self.dense5 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.flatten(x)
        
        # feature 추출
        x = self.dense1(x)
        x = self.batch1(x, training)
        x = tf.nn.relu(x)
        
        # DenseNet pre-activation
        h = self.batch2(x, training)
        h = tf.nn.relu(h)
        h = self.dense2(h)
        x = tf.concat([x, h], axis=-1)
        
        h = self.batch3(x, training)
        h = tf.nn.relu(h)
        h = self.dense3(h)
        x = tf.concat([x, h], axis=-1)
        
        h = self.batch4(x, training)
        h = tf.nn.relu(h)
        h = self.dense4(h)
        x = tf.concat([x, h], axis=-1)
        
        return self.dense5(x)

In [9]:
model = MyModel_DN()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/30
1875/1875 [==============================] - 7s 4ms/step - loss: 0.5165 - accuracy: 0.8161 - val_loss: 0.4446 - val_accuracy: 0.8444
Epoch 2/30
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4043 - accuracy: 0.8549 - val_loss: 0.3984 - val_accuracy: 0.8566
Epoch 3/30
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3662 - accuracy: 0.8663 - val_loss: 0.3804 - val_accuracy: 0.8610
Epoch 4/30
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3463 - accuracy: 0.8723 - val_loss: 0.4391 - val_accuracy: 0.8472
Epoch 5/30
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3283 - accuracy: 0.8784 - val_loss: 0.3785 - val_accuracy: 0.8625
Epoch 6/30
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3186 - accuracy: 0.8826 - val_loss: 0.3614 - val_accuracy: 0.8684
Epoch 7/30
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3105 - accuracy: 0.8846 - val_loss: 0.3694 - val_accuracy: